# IPHOTO ALBUM DOWNLOAD TOOL

#### This tool is designed to construct a local duplicate of a specified iCloud photo album. Upon generating the local copy, the tool systematically scans each image, compiling an inventory that includes any available GPS information. The resulting inventory, now populated with GPS coordinates, is stored in a local .csv file. This file can be parsed efficiently, offering an effective means to identify photos in close proximity to specified locations without the need to individually open each photo.

## START HERE TO LOAD MODULES AND BUILD SUB-ROUTINES

In [ ]:
from datetime import datetime
from exif import Image
import pyicloud
from pyicloud import PyiCloudService
import geopandas as gpd
import time
import csv

def icloud_initiate(username = "", password = ""):
    # Authenticate with your iCloud account
    api = PyiCloudService(username, password)
    if api.requires_2fa:
        print("Two-factor authentication required.")
        code = input("Enter the code you received of one of your approved devices: ")
        result = api.validate_2fa_code(code)
        print("Code validation result: %s" % result)
        if not result:
            print("Failed to verify security code")
            sys.exit(1)
        if not api.is_trusted_session:
            print("Session is not trusted. Requesting trust...")
            result = api.trust_session()
            print("Session trust result %s" % result)
            if not result:
                print("Failed to request trust. You will likely be prompted for the code again in the coming weeks")
    elif api.requires_2sa:
        import click
        print("Two-step authentication required. Your trusted devices are:")
        devices = api.trusted_devices
        for i, device in enumerate(devices):
            print(
                "  %s: %s" % (i, device.get('deviceName',
                "SMS to %s" % device.get('phoneNumber')))
            )
        device = click.prompt('Which device would you like to use?', default=0)
        device = devices[device]
        if not api.send_verification_code(device):
            print("Failed to send verification code")
            sys.exit(1)
        code = click.prompt('Please enter validation code')
        if not api.validate_verification_code(device, code):
            print("Failed to verify verification code")
            sys.exit(1)
    return(api)

def download_missing_icloud_photos(destination_folder = "", api=""):
    tic = time.perf_counter()
    photos = api.photos.all
    files_checked_count = 0
    new_files_found_count = 0
    file_list = []
    total_count = len(photos)
    local_file_list=[]
    for file in os.listdir(destination_folder):
        local_file_list.append(os.path.splitext(file)[0])
    for photo in photos:
        files_checked_count +=1
        file_extension = os.path.splitext(photo.filename)
        new_filename = format(photo.created.timestamp(),".3f") + file_extension[1]
        photo_filename = os.path.join(destination_folder, new_filename)
        if os.path.splitext(new_filename)[0] in local_file_list:
            pass
        else:
            new_files_found_count +=1
            with open(photo_filename, 'wb') as opened_file:
                opened_file.write(photo.download().content)
        print("New iPhotos found on icloud = " + str(new_files_found_count) + " (Checking: " + str(files_checked_count) + " of " + str(total_count) + ")", end ="\r")
    print("\nMissing photos download done! Now run 'quick_photo_csv' or 'full_photo_csv'.")
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return()

def download_new_icloud_photos(destination_folder = "", api=""):
    photos = api.photos.all
    files_checked_count = 0
    new_files_found_count = 0
    file_list = []
    total_count = len(photos)
    local_file_list=[]
    for file in os.listdir(destination_folder):
        local_file_list.append(os.path.splitext(file)[0])
    for photo in photos:
        files_checked_count +=1
        file_extension = os.path.splitext(photo.filename)
        new_filename = format(photo.created.timestamp(),".3f") + file_extension[1]
        photo_filename = os.path.join(destination_folder, new_filename)
        if os.path.splitext(new_filename)[0] in local_file_list:
            pass
        else:
            new_files_found_count +=1
            with open(photo_filename, 'wb') as opened_file:
                opened_file.write(photo.download().content)
        print("New iPhotos found on icloud = " + str(new_files_found_count) + " (Checking: " + str(files_checked_count) + " of " + str(total_count) + ")", end ="\r")
        if files_checked_count - new_files_found_count > 20:
            break
    print("\nNew photos download done! Now run 'quick_photo_csv' or 'full_photo_csv'.")
    return()

def download_all_icloud_photos_again(destination_folder = "", api=""):
    tic = time.perf_counter()
    photos = api.photos.all
    files_downloaded_count = 0
    total_count = len(photos)
    for photo in photos:
        files_downloaded_count +=1
        file_extension = os.path.splitext(photo.filename)
        new_filename = format(photo.created.timestamp(),".3f") + file_extension[1]
        photo_filename = os.path.join(destination_folder, new_filename)
        with open(photo_filename, 'wb') as opened_file:
            opened_file.write(photo.download().content)
        print("iPhotos downloaded from icloud = " + str(files_downloaded_count) + " of " + str(total_count) + ")", end ="\r")
    print("\niCloud photos full download complete! Now run 'quick_photo_csv' or 'full_photo_csv'.")
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return()

def quick_photo_csv(destination_folder = ""):
    tic = time.perf_counter()
    photo_list_file = os.path.join(destination_folder, "list_of_photos.csv")
    file_list = []
    file_list_name = []
    if os.path.exists(photo_list_file) != True:
        file_list=[["File Name", "File type", "Device", "Date", "Lattitude", "Longitude"]]
    else:
        with open(photo_list_file, 'r') as file:
            csvreader = csv.reader(file)
            for row in csvreader:
                file_list.append(row)
                file_list_name.append(row[0])
    print("Opened 'List_of_photos.csv'. Looking for missed records.")
    files_checked_count = 0
    new_files_found = 0
    photo_quantity = len(os.listdir(destination_folder))
    for photo in os.listdir(destination_folder):
        files_checked_count +=1
        if os.path.splitext(photo)[0] in file_list_name:
            pass
        else:
            if os.path.splitext(photo)[1] == ".JPG" or os.path.splitext(photo)[1] == ".MOV":
                new_files_found +=1
                image_exif = get_image_coordinates(destination_folder + "\\" + photo)
                date = image_exif[0]
                coords_lat = image_exif[1]
                coords_long = image_exif[2]
                new_file=[os.path.splitext(photo)[0],os.path.splitext(photo)[1],"iPhoto", date, coords_lat, coords_long]
                file_list.append(new_file)
        print("Checking file " + str(files_checked_count) + " of " + str(photo_quantity) + ". New JPG or MOV files found = " + str(new_files_found), end ="\r")
    with open(photo_list_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(file_list)
    print("\nUpdated photo register written to 'List_of_photos.csv'.")
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return()

def full_photo_csv(destination_folder = ""):
    tic = time.perf_counter()
    print("Proceeding to gather info on disk contents. This will take a while.")
    files_checked_count =0
    file_list=[["File Name", "File type", "Device", "Date", "Lattitude", "Longitude"]]
    photo_quantity = len(os.listdir(destination_folder))
    for photo in os.listdir(destination_folder):
        files_checked_count +=1
        if os.path.splitext(photo)[1] == ".JPG" or os.path.splitext(photo)[1] == ".MOV":
            image_exif = get_image_coordinates(destination_folder + "\\" + photo)
            date = image_exif[0]
            coords_lat = image_exif[1]
            coords_long = image_exif[2]
            new_file=[os.path.splitext(photo)[0],os.path.splitext(photo)[1],"iPhoto", date, coords_lat, coords_long]
            file_list.append(new_file)
        print("Photos resolved = " + str(files_checked_count) + " of " + str(photo_quantity), end ="\r")
    photo_list_file = os.path.join(destination_folder, "list_of_photos.csv")
    with open(photo_list_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(file_list)
    print("\nNew photo register written to 'List_of_photos.csv'.")
    toc = time.perf_counter()
    print(f"Time taken: {(toc - tic)/60:0.4f} minutes")
    return()

def get_image_coordinates(image_file):
    date = 0
    coords_lat = 0
    coords_long = 0
    try:
        with open(image_file, 'rb') as src:
            img = Image(src)
            date=('')
            coords=(0,0)
            if img.has_exif:
                try:
                    date = img.datetime_original
                except:
                    pass
                try:
                    coords_lat = decimal_coords(img.gps_latitude,img.gps_latitude_ref)
                    coords_long = decimal_coords(img.gps_longitude,img.gps_longitude_ref)
                except AttributeError:
                    if os.path.splitext(image_file)[1] != ".MOV":
                        print("File may be corrupt: " + image_file)
                    pass
            else:
                pass
    except:
        if os.path.splitext(image_file)[1] != ".MOV":
            print("File may be corrupt: " + image_file)
        pass
    return (date, coords_lat, coords_long)

def decimal_coords(coords, ref):
    decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
    if ref == 'S' or ref == 'W':
        decimal_degrees = -decimal_degrees
    return decimal_degrees

## SET PHOTO FOLDER LOCATION AND iCLOUD LOGIN CREDENTIALS

In [ ]:
##Authenticate and initilise photo location
username = 'my_email@address'
password = 'my_password'
iphoto_folder_destination = "D:\\my_repository_folder"
os.path.exists(iphoto_folder_destination)
print("File location = " + iphoto_folder_destination + " - " + str(os.path.exists(iphoto_folder_destination)))

## COME HERE TO DOWNLOAD PHOTOS FROM iCLOUD

In [ ]:
#Download NEW photos from iPhoto album. Quickest!
#This can run in the background as the CSV file will not be updated automatically.
api = icloud_initiate(username = username, password = password) #Login to iCloud
download_new_icloud_photos(iphoto_folder_destination, api)

In [ ]:
#Download MISSING photos from iPhoto album. Not the quickest. This will a little time to complete!
#This can run in the background as the CSV file will not be updated automatically.
api = icloud_initiate(username = username, password = password) #Login to iCloud
download_missing_icloud_photos(iphoto_folder_destination, api)

In [ ]:
#Download FULL content of iPhoto album. This is SLOW!!! This will take many hours to complete!
#This can run in the background as the CSV file will not be updated automatically.
api = icloud_initiate(username = username, password = password) #Login to iCloud
download_all_icloud_photos_again(iphoto_folder_destination, api)

## COME HERE TO UPDATE .CSV PHOTO REGISTER

In [ ]:
#Check the photo register for missing photos. Quickest!
quick_photo_csv(iphoto_folder_destination)

In [ ]:
#Do a full refresh of the photo register. Not the quickest. This will take a couple hours!
full_photo_csv(iphoto_folder_destination)

## COME HERE TO PERFORM A TYPICAL 'NEW PHOTO'S DOWNLOAD AND A QUICK REGISTER UPDATE' (AFTER AUTHENTICATE IS DONE FIRST).

In [ ]:
api = icloud_initiate(username = username, password = password) #Login to iCloud
download_new_icloud_photos(iphoto_folder_destination, api)
quick_photo_csv(iphoto_folder_destination)